### RUNNING MODELS WITH OPTIMUM and ONNX Runtime

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM  #, pipeline
# from optimum.pipelines import pipeline
from optimum.onnxruntime import ORTModelForSeq2SeqLM, pipeline
import time
import accelerate


# model = AutoModelForSeq2SeqLM.from_pretrained("Falconsai/text_summarization")
model_name = "Rahmat82/t5-small-finetuned-summarization-xsum"
# model_name = "Falconsai/text_summarization"
# model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = ORTModelForSeq2SeqLM.from_pretrained(model_name, export=True)
summarizer = pipeline("summarization", model = model, tokenizer=tokenizer, device_map="auto", batch_size=12)


src = "A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here's a breakdown of its various aspects: Purpose The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit. Content A typical cover letter includes the following sections: Header: Includes your contact information, the date, and the employer's contact information. Salutation: A greeting to the hiring manager, preferably personalized with their name. Introduction: Briefly introduces who you are and the position you're applying for. Body: This is the core of your cover letter where you discuss your qualifications, experiences, and skills that make you suitable for the job. You can also mention how you can contribute to the company. Conclusion: Summarizes your points and reiterates your enthusiasm for the role. You can also include a call to action, like asking for an interview. Signature: A polite closing ('Sincerely,' 'Best regards,' etc.) followed by your name."

gold = "A cover letter is a professional note sent with your resume when applying for a role. It introduces you to the hiring manager and gives additional context beyond what your resume shows. Its purpose is to highlight your key skills, relevant experience, enthusiasm for the position, and reasons you're a strong match. A standard cover letter contains several parts: a header with your details and the employer's information, a personalized greeting, a short introduction about who you are and the job you want, a main body explaining your qualifications and how you can help the company, and a closing that reinforces your interest and politely asks for the next step. It typically ends with a formal sign-off such as 'Sincerely' followed by your name."


# params = {'max_length': 150}
# params['min_length'] = 100
# params['do_sample'] = False


# params['num_beams'] = 2
# params['early_stopping']=True
# params['no_repeat_ngram_size']=2

start = time.perf_counter()
try:
    pred_obj = summarizer(src)
    pred = pred_obj[0].get('summary_text') if isinstance(pred_obj, list) and len(pred_obj) > 0 else str(pred_obj)
except Exception as e:
    pred = None
    print(f"Error: {e}")
latency_ms = (time.perf_counter() - start) * 1000.0

rec = {
    'source': src,
    'gold': gold,
    'pred': pred,
    'latency_ms': latency_ms,
}
print(rec)
print(f"latency {latency_ms:.1f} ms")

Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
c:\Users\parag\anaconda3\envs\summarizer\Lib\site-packages\transformers\modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask
c:\Users\parag\anaconda3\envs\summarizer\Lib\site-packages\transformers\integrations\sdpa_attention.py:82: TracerWarning: Converting a tensor to a Python boolean might ca

{'source': "A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here's a breakdown of its various aspects: Purpose The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit. Content A typical cover letter includes the following sections: Header: Includes your contact information, the date, and the employer's contact information. Salutation: A greeting to the hiring manager, preferably personalized with their name. Introduction: Briefly introduces who you are and the position you're applying for. Body: This is the core of your cover letter where you discuss

In [19]:
pred

"A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role."

### QUANTIING ORT SEQ2SEQ MODEL

In [6]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM, pipeline

# model_name = "Rahmat82/t5-small-finetuned-summarization-xsum"
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = ORTModelForSeq2SeqLM.from_pretrained(model_name, export=True)

Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
c:\Users\parag\anaconda3\envs\summarizer\Lib\site-packages\transformers\modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask
c:\Users\parag\anaconda3\envs\summarizer\Lib\site-packages\transformers\integrations\sdpa_attention.py:82: TracerWarning: Converting a tensor to a Python boolean might ca

In [7]:
from optimum.onnxruntime import ORTQuantizer, ORTModelForSeq2SeqLM
from optimum.onnxruntime.configuration import AutoQuantizationConfig

model_dir = model.model_save_dir
print(model_dir)
# Create encoder quantizer
encoder_quantizer = ORTQuantizer.from_pretrained(model_dir, file_name="encoder_model.onnx")

# Create decoder quantizer
decoder_quantizer = ORTQuantizer.from_pretrained(model_dir, file_name="decoder_model.onnx")

# Create decoder with past key values quantizer
decoder_wp_quantizer = ORTQuantizer.from_pretrained(model_dir, file_name="decoder_with_past_model.onnx")

# Create Quantizer list
quantizer = [encoder_quantizer, decoder_quantizer, decoder_wp_quantizer]

C:\Users\parag\AppData\Local\Temp\tmpzpdmhnds


c:\Users\parag\anaconda3\envs\summarizer\Lib\site-packages\transformers\models\bart\configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [8]:
# Define the quantization strategy by creating the appropriate configuration
dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)
# Quantize the models individually
for q in quantizer:
    # q.quantize(save_dir="../models/quantized_models/t5-small-finetuned-summarization-xsum",quantization_config=dqconfig)
    q.quantize(save_dir="../models/quantized_models/bart-large-cnn",quantization_config=dqconfig)
    

c:\Users\parag\anaconda3\envs\summarizer\Lib\site-packages\transformers\configuration_utils.py:448: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'forced_bos_token_id': 0}
  warnings.warn(


### LOADING THE QUANTIZED ORT SEQ2SEQ MODEL

In [12]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM  #, pipeline
# from optimum.pipelines import pipeline
from optimum.onnxruntime import ORTModelForSeq2SeqLM, pipeline
import time
import accelerate
# Path to the directory where the quantized model is saved
# quantized_model_dir = "../models/quantized_models/t5-small-finetuned-summarization-xsum/"
quantized_model_dir = "../models/quantized_models/bart-large-cnn/"

quantized_model = ORTModelForSeq2SeqLM.from_pretrained(quantized_model_dir)
quantized_tokenizer = AutoTokenizer.from_pretrained(quantized_model_dir)
summarizer = pipeline("summarization", model=quantized_model, tokenizer=quantized_tokenizer, device_map="auto", batch_size=12)

Could not find any ONNX files with standard file name decoder_model_merged.onnx, files found: [WindowsPath('decoder_model.onnx'), WindowsPath('decoder_with_past_model.onnx'), WindowsPath('encoder_model.onnx')]. Please make sure to pass a `file_name` and/or `subfolder` argument to `from_pretrained` when loading an ONNX file with non-standard file names.
Moving the following attributes in the config to the generation config: {'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
Device set to use cpu


In [16]:
src = "A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here's a breakdown of its various aspects: Purpose The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit. Content A typical cover letter includes the following sections: Header: Includes your contact information, the date, and the employer's contact information. Salutation: A greeting to the hiring manager, preferably personalized with their name. Introduction: Briefly introduces who you are and the position you're applying for. Body: This is the core of your cover letter where you discuss your qualifications, experiences, and skills that make you suitable for the job. You can also mention how you can contribute to the company. Conclusion: Summarizes your points and reiterates your enthusiasm for the role. You can also include a call to action, like asking for an interview. Signature: A polite closing ('Sincerely,' 'Best regards,' etc.) followed by your name."

gold = "A cover letter is a professional note sent with your resume when applying for a role. It introduces you to the hiring manager and gives additional context beyond what your resume shows. Its purpose is to highlight your key skills, relevant experience, enthusiasm for the position, and reasons you're a strong match. A standard cover letter contains several parts: a header with your details and the employer's information, a personalized greeting, a short introduction about who you are and the job you want, a main body explaining your qualifications and how you can help the company, and a closing that reinforces your interest and politely asks for the next step. It typically ends with a formal sign-off such as 'Sincerely' followed by your name."

params = {}
params = {'max_length': 100}
# params = {'min_length': 100}
params = {'do_sample': False}
# params['num_beams'] = 2
# params['early_stopping']=True
# params['no_repeat_ngram_size']=2

start = time.perf_counter()
try:
    pred_obj = summarizer(src, **params)
    pred = pred_obj[0].get('summary_text') if isinstance(pred_obj, list) and len(pred_obj) > 0 else str(pred_obj)
except Exception as e:
    pred = None
    print(f"Error: {e}")
latency_ms = (time.perf_counter() - start) * 1000.0

rec = {
    'source': src,
    'gold': gold,
    'pred': pred,
    'latency_ms': latency_ms,
}
print(rec)
print(f"latency {latency_ms:.1f} ms")

{'source': "A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here's a breakdown of its various aspects: Purpose The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit. Content A typical cover letter includes the following sections: Header: Includes your contact information, the date, and the employer's contact information. Salutation: A greeting to the hiring manager, preferably personalized with their name. Introduction: Briefly introduces who you are and the position you're applying for. Body: This is the core of your cover letter where you discuss

In [17]:
pred

'A cover letter is a formal document that accompanies your resume when you apply for a job. A cover letter in the form of a cover letter, or a cover sheet, in the job of a job in the workplace. It is a work of the work of a work in the work in a job, or the work, of the job, and the job in a workplace. A work, a work, and a work work, the cover letter and the work that you have in your resume. The cover letter of the cover of the magazine, a magazine, the magazine of the newspaper, a newspaper, the newspaper and a newspaper. The advertisement of the advertisement, the advertisement of an advertisement, and an advertisement in the advertising of the advertising in the newspaper.'

### MODEL CHOICE EXPERIMENTS for HUMAN VALIDATED ACCURACY AND LATENCY

In [ ]:
import time
from transformers import pipeline
# from optimum.onnxruntime import pipeline


# model_name = 'eugenesiow/bart-paraphrase'
# model_name = 'Xenova/bart-large-cnn'
# model_name = 'Falconsai/medical_summarization'
model_name = 'Falconsai/text_summarization'
# model_name = 'Xenova/distilbart-xsum-12-1'
summarizer = pipeline('summarization', model=model_name)


src = "A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here's a breakdown of its various aspects: Purpose The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit. Content A typical cover letter includes the following sections: Header: Includes your contact information, the date, and the employer's contact information. Salutation: A greeting to the hiring manager, preferably personalized with their name. Introduction: Briefly introduces who you are and the position you're applying for. Body: This is the core of your cover letter where you discuss your qualifications, experiences, and skills that make you suitable for the job. You can also mention how you can contribute to the company. Conclusion: Summarizes your points and reiterates your enthusiasm for the role. You can also include a call to action, like asking for an interview. Signature: A polite closing ('Sincerely,' 'Best regards,' etc.) followed by your name."

gold = "A cover letter is a professional note sent with your resume when applying for a role. It introduces you to the hiring manager and gives additional context beyond what your resume shows. Its purpose is to highlight your key skills, relevant experience, enthusiasm for the position, and reasons you're a strong match. A standard cover letter contains several parts: a header with your details and the employer's information, a personalized greeting, a short introduction about who you are and the job you want, a main body explaining your qualifications and how you can help the company, and a closing that reinforces your interest and politely asks for the next step. It typically ends with a formal sign-off such as 'Sincerely' followed by your name."


params = {'max_length': 150}
params = {'min_length': 100}
params['do_sample'] = False
# params['num_beams'] = 2
# params['early_stopping']=True
# params['no_repeat_ngram_size']=2

start = time.perf_counter()
try:
    pred_obj = summarizer(src, **params)
    pred = pred_obj[0].get('summary_text') if isinstance(pred_obj, list) and len(pred_obj) > 0 else str(pred_obj)
except Exception as e:
    pred = None
    print(f"Error: {e}")
latency_ms = (time.perf_counter() - start) * 1000.0

rec = {
    'source': src,
    'gold': gold,
    'pred': pred,
    'latency_ms': latency_ms,
}
print(rec)
print(f"latency {latency_ms:.1f} ms")

Device set to use cpu


{'source': "A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here's a breakdown of its various aspects: Purpose The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit. Content A typical cover letter includes the following sections: Header: Includes your contact information, the date, and the employer's contact information. Salutation: A greeting to the hiring manager, preferably personalized with their name. Introduction: Briefly introduces who you are and the position you're applying for. Body: This is the core of your cover letter where you discuss

In [15]:
import torch
from transformers import pipeline

pipeline = pipeline(
    task="summarization",
    model="google/pegasus-xsum"
)


src = "A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here's a breakdown of its various aspects: Purpose The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit. Content A typical cover letter includes the following sections: Header: Includes your contact information, the date, and the employer's contact information. Salutation: A greeting to the hiring manager, preferably personalized with their name. Introduction: Briefly introduces who you are and the position you're applying for. Body: This is the core of your cover letter where you discuss your qualifications, experiences, and skills that make you suitable for the job. You can also mention how you can contribute to the company. Conclusion: Summarizes your points and reiterates your enthusiasm for the role. You can also include a call to action, like asking for an interview. Signature: A polite closing ('Sincerely,' 'Best regards,' etc.) followed by your name."
gold = "A cover letter is a professional note sent with your resume when applying for a role. It introduces you to the hiring manager and gives additional context beyond what your resume shows. Its purpose is to highlight your key skills, relevant experience, enthusiasm for the position, and reasons you're a strong match. A standard cover letter contains several parts: a header with your details and the employer's information, a personalized greeting, a short introduction about who you are and the job you want, a main body explaining your qualifications and how you can help the company, and a closing that reinforces your interest and politely asks for the next step. It typically ends with a formal sign-off such as 'Sincerely' followed by your name."


params = {'max_new_tokens': 150}
params['do_sample'] = False


start = time.perf_counter()
try:
    pred_obj = summarizer(src, **params)
    pred = pred_obj[0].get('summary_text') if isinstance(pred_obj, list) and len(pred_obj) > 0 else str(pred_obj)
except Exception as e:
    pred = None
    print(f"Error: {e}")
latency_ms = (time.perf_counter() - start) * 1000.0

rec = {
    'source': src,
    'gold': gold,
    'pred': pred,
    'latency_ms': latency_ms,
}
print(rec)
print(f"latency {latency_ms:.1f} ms")

c:\Users\parag\anaconda3\envs\summarizer\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\parag\.cache\huggingface\hub\models--google--pegasus-xsum. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTT

{'source': "A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here's a breakdown of its various aspects: Purpose The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit. Content A typical cover letter includes the following sections: Header: Includes your contact information, the date, and the employer's contact information. Salutation: A greeting to the hiring manager, preferably personalized with their name. Introduction: Briefly introduces who you are and the position you're applying for. Body: This is the core of your cover letter where you discuss

In [ ]:
import time
from transformers import pipeline

model_name = 'Xenova/distilbart-xsum-12-1'
summarizer = pipeline('summarization', model=model_name)

src = "A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here's a breakdown of its various aspects: Purpose The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit. Content A typical cover letter includes the following sections: Header: Includes your contact information, the date, and the employer's contact information. Salutation: A greeting to the hiring manager, preferably personalized with their name. Introduction: Briefly introduces who you are and the position you're applying for. Body: This is the core of your cover letter where you discuss your qualifications, experiences, and skills that make you suitable for the job. You can also mention how you can contribute to the company. Conclusion: Summarizes your points and reiterates your enthusiasm for the role. You can also include a call to action, like asking for an interview. Signature: A polite closing ('Sincerely,' 'Best regards,' etc.) followed by your name."
gold = "A cover letter is a professional note sent with your resume when applying for a role. It introduces you to the hiring manager and gives additional context beyond what your resume shows. Its purpose is to highlight your key skills, relevant experience, enthusiasm for the position, and reasons you're a strong match. A standard cover letter contains several parts: a header with your details and the employer's information, a personalized greeting, a short introduction about who you are and the job you want, a main body explaining your qualifications and how you can help the company, and a closing that reinforces your interest and politely asks for the next step. It typically ends with a formal sign-off such as 'Sincerely' followed by your name."

params = {'max_new_tokens': 256}
params['do_sample'] = False

start = time.perf_counter()
try:
    pred_obj = summarizer(src, **params)
    pred = pred_obj[0].get('summary_text') if isinstance(pred_obj, list) and len(pred_obj) > 0 else str(pred_obj)
except Exception as e:
    pred = None
    print(f"Error: {e}")
latency_ms = (time.perf_counter() - start) * 1000.0

rec = {
    'source': src,
    'gold': gold,
    'pred': pred,
    'latency_ms': latency_ms,
}
print(rec)
print(f"latency {latency_ms:.1f} ms")

In [1]:
import time
from transformers import AutoTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

c:\Users\parag\anaconda3\envs\summarizer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
src = "A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here's a breakdown of its various aspects: Purpose The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit. Content A typical cover letter includes the following sections: Header: Includes your contact information, the date, and the employer's contact information. Salutation: A greeting to the hiring manager, preferably personalized with their name. Introduction: Briefly introduces who you are and the position you're applying for. Body: This is the core of your cover letter where you discuss your qualifications, experiences, and skills that make you suitable for the job. You can also mention how you can contribute to the company. Conclusion: Summarizes your points and reiterates your enthusiasm for the role. You can also include a call to action, like asking for an interview. Signature: A polite closing ('Sincerely,' 'Best regards,' etc.) followed by your name."
gold = "A cover letter is a professional note sent with your resume when applying for a role. It introduces you to the hiring manager and gives additional context beyond what your resume shows. Its purpose is to highlight your key skills, relevant experience, enthusiasm for the position, and reasons you're a strong match. A standard cover letter contains several parts: a header with your details and the employer's information, a personalized greeting, a short introduction about who you are and the job you want, a main body explaining your qualifications and how you can help the company, and a closing that reinforces your interest and politely asks for the next step. It typically ends with a formal sign-off such as 'Sincerely' followed by your name."

inputs = tokenizer([src], max_length=1024, return_tensors="pt")

# Generate Summary
start = time.perf_counter()
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=100, max_length=200)
pred = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
latency_ms = (time.perf_counter() - start) * 1000.0


rec = {
    'source': src,
    'gold': gold,
    'pred': pred,
    'latency_ms': latency_ms,
}
print(rec)
print(f"latency {latency_ms:.1f} ms")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'source': "A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here's a breakdown of its various aspects: Purpose The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It's also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit. Content A typical cover letter includes the following sections: Header: Includes your contact information, the date, and the employer's contact information. Salutation: A greeting to the hiring manager, preferably personalized with their name. Introduction: Briefly introduces who you are and the position you're applying for. Body: This is the core of your cover letter where you discuss